This is an example of "ungeocoding" a geotiff to radar coordinates. It is based on how ISCE2 geocodes radar files. This assumes you want to resample to full resolution, but this can also be done to downlooked coordinates.

---

In [ ]:
import argparse
import isce
#import isceobj
import os
import shutil
from osgeo import gdal
from osgeo import gdal_array
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
#import rasterio
#from rasterio.plot import show
import glob
#import matplotlib
#matplotlib.use('Agg')
#import matplotlib.pyplot as plt

In [ ]:
# Following files need to be the same size as your output grid.
# lat and lon rdr files in geom_reference dir from stack processor
# this is for full res, change if you want downlooked
latfile  = 'geom_reference/lat.rdr.full.vrt'
lonfile  = 'geom_reference/lon.rdr.full.vrt'

In [ ]:
# setup paths and get input files
resampMethod = 'near'
workdir      = 'workdir' # path to your working dir
origdir      = f'{workdir}/original' # path to your original tiff dir
##os.mkdir(original) # needs to exist and be empty
radardir = f'{workdir}/radar' # where your radar coord files will go
#os.mkdir(cropradardir) # needs to exist and be empty
infiles      = glob.glob(f'{origdir}/*.tif')
infiles      = sorted(infiles)
infile       = infiles[0] # file for getting bounds, data type.  For all Landsat 8 tiffs
os.chdir(workdir) # change to cropscape dir

In [ ]:
def writeVRT(infile, latFile, lonFile):
#This function is modified from isce2gis.py
            tree = ET.parse(infile + '.vrt')
            root = tree.getroot()

            meta = ET.SubElement(root, 'metadata')
            meta.attrib['domain'] = "GEOLOCATION"
            meta.tail = '\n'
            meta.text = '\n    '


            rdict = { 'Y_DATASET' : latFile,
                      'X_DATASET' : lonFile,
                      'X_BAND' : "1",
                      'Y_BAND' : "1",
                      'PIXEL_OFFSET': "0",
                      'LINE_OFFSET' : "0",
                      'LINE_STEP' : "1",
                      'PIXEL_STEP' : "1" }

            for key, val in rdict.items():
                data = ET.SubElement(meta, 'mdi')
                data.text = val
                data.attrib['key'] = key
                data.tail = '\n    '

            data.tail = '\n'
            tree.write(infile + '.vrt')

In [ ]:
###figure out bounds of lat/lonfile and crop input file
lf = gdal.Open(latfile)
stats = lf.GetRasterBand(1).GetStatistics(0,1)
sarMinLat = stats[0]
sarMaxLat = stats[1]
lf=None

###figure out bounds of lat/lonfile and crop input file
lf = gdal.Open(lonfile)
stats = lf.GetRasterBand(1).GetStatistics(0,1)
sarMinLon = stats[0]
sarMaxLon = stats[1]
lf=None

###get pixel spacing on infile
lf    = gdal.Open(infile)
stats = lf.GetGeoTransform()
dLon  = stats[1]
dLat  = -stats[5]
band = lf.GetRasterBand(1)
arr = band.ReadAsArray()
dtype = gdal.GetDataTypeName(band.DataType)
print(dtype)
#plt.imshow(arr)
#plt.show()
lf   = None


In [ ]:
##get number of rows and columns in radar coords file (lat)
raster = gdal.Open(latfile)
radarnx=raster.RasterXSize
radarny=raster.RasterYSize
cols,rows=np.meshgrid(np.arange(1,radarnx+1),np.arange(radarny,0,-1))

#make new files of columns and rows, in radar coordinates
colfile = 'colsfr.r4'
rowfile = 'rowsfr.r4'
gcolfile = 'geo_colsfr.r4'
growfile = 'geo_rowsfr.r4'
driver=gdal.GetDriverByName('ISCE')


In [ ]:
colds = driver.Create(colfile,radarnx,radarny,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(cols)
colds=None

colds = driver.Create(rowfile,radarnx,radarny,1,gdal.GDT_Float32)
colds.GetRasterBand(1).WriteArray(rows)
colds=None

cmd = 'fixImageXml.py -i '+colfile+' -f'
os.system(cmd)
cmd = 'fixImageXml.py -i '+rowfile+' -f'
os.system(cmd)

writeVRT(colfile, latfile, lonfile)
writeVRT(rowfile, latfile, lonfile)

In [ ]:
##get number of rows and columns + min/max range of input file
raster = gdal.Open(infile)
geonx=raster.RasterXSize
geony=raster.RasterYSize

minLon = raster.GetGeoTransform()[0]
deltaLon = raster.GetGeoTransform()[1]
maxLat = raster.GetGeoTransform()[3]
deltaLat = raster.GetGeoTransform()[5]
minLat = maxLat + geony*deltaLat
maxLon = minLon + geonx*deltaLon
WSEN = str(minLon)+' '+str(minLat)+' '+str(maxLon)+' '+str(maxLat)


In [ ]:
#geocode row and col file
cmd = 'gdalwarp -of ISCE -geoloc  -te '+WSEN+' -tr '+str(deltaLon)+' '+str(deltaLat)+' -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear ' +rowfile +'.vrt ' + growfile
print(cmd)
os.system(cmd)
cmd = 'gdalwarp -of ISCE -geoloc  -te '+WSEN+' -tr '+str(deltaLon)+' '+str(deltaLat)+' -srcnodata 0 -dstnodata 0  -wt float32 -r bilinear ' +colfile +'.vrt ' + gcolfile
print(cmd)
os.system(cmd)
cmd = 'gdalbuildvrt '+gcolfile+'.vrt '+gcolfile
os.system(cmd)
cmd = 'gdalbuildvrt '+growfile+'.vrt '+growfile
os.system(cmd)

In [ ]:
###set any geocoding info
WSEN = '0.5 0.5 '+str(radarnx+0.5)+' '+str(radarny+0.5)
for file in infiles:
    outfile  = f'{radardir}/{os.path.basename(file)}'
    writeVRT(file, growfile, gcolfile)
    cmd = 'gdalwarp -geoloc -of ISCE -te '+ WSEN + ' -tr 1 1 -srcnodata 0 -dstnodata 0 -wt '+dtype+' -r ' + resampMethod + ' ' + file +'.vrt ' +outfile
    os.system(cmd)
    cmd = 'gdalbuildvrt '+outfile+'.vrt '+outfile
    os.system(cmd)
    os.rename(outfile,f'{radardir}/{file[50:58]}.tif')
    os.rename(f'{outfile}.vrt',f'{radardir}/{file[50:58]}.tif.vrt')
    os.rename(f'{outfile}.xml',f'{radardir}/{file[50:58]}.tif.xml')
    os.rename(f'{outfile}.aux.xml',f'{radardir}/{file[50:58]}.tif.aux.xml')